### Описание гипотезы
Проверим два предположения:
- Зависит ли вероятность оплаты от выбранного пользователем уровня сложности бесплатных тренировок?
- Существует ли разница во времени между пользователями с разным уровнем сложности и их первой оплатой?

Проверку будем производить на основе данных пользователей, которые зарегистрировались в 2017 году.

### Ход проверки

Импортируем нужные библиотеки

In [3]:
import pandas as pd
import numpy as np

Получаем данные из базы данных

In [4]:
events_df = pd.read_csv('gd3/events.csv', sep=',')
purchase_df = pd.read_csv('gd3/purchase.csv', sep=',')

Произведем отбор пользователей, зарегистрировавшихся в 2018 году, а также переведем столбцы с датой в нужный формат

In [35]:
cond = (events_df.start_time>='2018-01-01') & (events_df.start_time<'2019-01-01') & (events_df.event_type=='registration')
registered = events_df[cond]['user_id'].to_list() # список пользователей, зарег. в 2018
events = events_df[events_df.user_id.isin(registered)]
events.start_time = pd.to_datetime(events.start_time, format='%Y-%m-%dT%H:%M:%S')

cond_2 = (purchase_df.event_datetime>='2018-01-01') & (purchase_df.event_datetime<'2019-01-01') & (events_df.event_type=='registration')
purchase = purchase_df[purchase_df.user_id.isin(registered)]
purchase.event_datetime = pd.to_datetime(purchase.event_datetime, format='%Y-%m-%dT%H:%M:%S')

C:\Users\nason\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Перед объединением датафреймов, переименуем колонки

In [36]:
events_df = events.rename(columns={'id':'event_id'})
purchase_df = purchase.rename(columns={'id':'purchase_id'})

Сделаем объединенный датафрейм из событий и оплат

In [37]:
total_events_df = pd.concat([events_df,purchase_df],sort=False)
total_events_df.info(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68559 entries, 51405 to 2778
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        66959 non-null  float64       
 1   event_type      66959 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         68559 non-null  int64         
 6   purchase_id     1600 non-null   float64       
 7   event_datetime  1600 non-null   datetime64[ns]
 8   amount          1600 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 5.2+ MB


Сделаем группы пользователей по уровню сложности, который был выбран для тренировок.

Сначала посмотрим как уровни сложности бывают:

In [38]:
total_events_df['selected_level'].unique()

array([nan, 'medium', 'hard', 'easy'], dtype=object)

Сформируем группы

In [39]:
users_easy_level = total_events_df[total_events_df['selected_level'] == 'easy']['user_id'].unique()
users_medium_level = total_events_df[total_events_df['selected_level'] == 'medium']['user_id'].unique()
users_hard_level = total_events_df[total_events_df['selected_level'] == 'hard']['user_id'].unique()

Посчитаем для каждой группы показатели

In [41]:
user_groups = {'easy': users_easy_level,
               'medium': users_medium_level,
               'hard': users_hard_level}

for group in user_groups:
    count_of_users_in_group = purchase_df['user_id'].nunique()
    purchase_df_slice = purchase_df[purchase_df['user_id'].isin(user_groups[group])]
    percent_of_purchase = purchase_df_slice['user_id'].nunique()/count_of_users_in_group
    print('Процент пользователей, выбравших уровень сложности {}: {:.2%}'.format(group, percent_of_purchase))
    print()
    level_choice_df = total_events_df[(total_events_df['event_type'] == 'level_choice') & 
                                     (total_events_df['user_id'].isin(user_groups[group]))]
    if (level_choice_df['user_id'].value_counts().mean()) == 1:
        level_choice_df = level_choice_df[['user_id','start_time']].rename(columns = {'start_time':'level_choice_time'})
        purchase_df_slice_2 = purchase_df_slice[['user_id','event_datetime']].rename(columns = {'event_datetime':'purchase_time'})
        merged_df = purchase_df_slice_2.merge(level_choice_df, on = 'user_id', how = 'inner')
        merged_df['timedelta'] = merged_df['purchase_time'] - merged_df['level_choice_time']
        mean_time = merged_df['timedelta'].mean()
        print('Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности {}: {}'.format(group, mean_time))
        print('Характеристики времени:')
        print((merged_df['timedelta'].describe()))
        print()
    else:
        print('Более 1 события выбора уровня сложности')


Процент пользователей, выбравших уровень сложности easy: 11.81%

Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности easy: 3 days 14:58:52.941798941
Характеристики времени:
count                          189
mean     3 days 14:58:52.941798941
std      2 days 07:06:35.644097504
min                0 days 00:49:20
25%                1 days 17:18:56
50%                3 days 06:03:50
75%                5 days 06:58:18
max               10 days 18:35:09
Name: timedelta, dtype: object

Процент пользователей, выбравших уровень сложности medium: 60.56%

Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности medium: 3 days 23:14:13.165118679
Характеристики времени:
count                          969
mean     3 days 23:14:13.165118679
std      2 days 06:18:57.618467109
min                0 days 04:18:12
25%                2 days 01:20:07
50%                3 days 19:53:19
75%                5 days 16

### Выводы
Процент оплативших пользователей, выбравших уровень сложности easy: 11.81%
Процент оплативших пользователей, выбравших уровень сложности medium: 60.56%
Процент оплативших пользователей, выбравших уровень сложности hard: 27.62%

Таким образом, нет прямой зависимости вероятности оплаты от выбранного уровня сложности.
Пользователи, выбирающие тренировки средней сложности, чаще покупают платные тренировки.

Наибольшее время между событием выбора уровня сложности и моментом первой оплаты, тратят пользователи, которые выбрали уровень сложности medium. Для них среднее время составляет 3 дня 23 часа, а медианное время 3 дня 19 часов.

Чуть меньше времени тратят пользователи, выбравшие уровень сложности easy. Для них среднее время составляет 3 дня 14 часов, а медианное время 3 дня 6 часов.

Меньше всего времени на принятие решение о покупке тратят пользователя, выбравшие уровень сложности hard. Для них среднее время составляет 3 дня 7 часов, а медианное время 3 дня 13 часов.